In [1]:
import h5py
import torch
import time
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# with h5py.File('data/KdV_train_1024_default.h5', 'r') as f:
#     traj_train = torch.tensor(f['train']['pde_140-256'][:], dtype=torch.float32)
# with h5py.File('data/KdV_valid_1024_default.h5', 'r') as f:
#     traj_valid = torch.tensor(f['valid']['pde_140-256'][:], dtype=torch.float32)
# with h5py.File('data/KdV_test_4096_default.h5', 'r') as f:
#     traj_test = torch.tensor(f['test']['pde_140-256'][:], dtype=torch.float32)

class args:
    equation = 'KdV'
    # equation = 'Burgers'

class Traj_dataset:
    traj_train = None
    traj_valid = None
    traj_test = None

# with h5py.File(f'data/{args.equation}_train_1024_default.h5', 'r') as f:
#     Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:], dtype=torch.float32)[:, :131]
# with h5py.File(f'data/{args.equation}_valid_1024_default.h5', 'r') as f:
#     Traj_dataset.traj_valid = torch.tensor(f['valid']['pde_140-256'][:], dtype=torch.float32)[:, :131]
# with h5py.File(f'data/{args.equation}_test_4096_default.h5', 'r') as f:
#     Traj_dataset.traj_test = torch.tensor(f['test']['pde_140-256'][:], dtype=torch.float32)[:, :131]

print('Loading training data...')
with h5py.File(f'data_large/{args.equation}_train_100000_default.h5', 'r') as f:
    # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:10000, :131], dtype=torch.float32, device=cfg.device)
    Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:1000, :10], dtype=torch.float32)
    # Traj_dataset.traj_train = torch.tensor(f['train']['pde_140-256'][:100, :131], dtype=torch.float32, device=cfg.device)
# print('Loading validation data...')
# with h5py.File(f'data_large/{cfg.equation}_valid_1024_default.h5', 'r') as f:
#     Traj_dataset.traj_valid = torch.tensor(f['valid']['pde_140-256'][:, :131], dtype=torch.float32)
print('Loading test data...')
with h5py.File(f'data_large/{args.equation}_test_100000_default.h5', 'r') as f:
    # Traj_dataset.traj_test = torch.tensor(f['test']['pde_140-256'][:, :131], dtype=torch.float32)
    Traj_dataset.traj_test = torch.tensor(f['test']['pde_140-256'][:10000, :10], dtype=torch.float32)

Loading training data...
Loading test data...


In [3]:
from typing import Any, Dict, List, Tuple
import torch.nn.functional as F

class LpLoss(object):
    def __init__(self, d=2, p=2, size_average=True, reduction=True):
        super(LpLoss, self).__init__()

        #Dimension and Lp-norm type are postive
        assert d > 0 and p > 0

        self.d = d
        self.p = p
        self.reduction = reduction
        self.size_average = size_average

    def abs(self, x, y):
        num_examples = x.size()[0]

        #Assume uniform mesh
        h = 1.0 / (x.size()[1] - 1.0) if x.size()[1] > 1 else 1.0

        all_norms = (h**(self.d/self.p))*torch.norm(x.view(num_examples,-1) - y.view(num_examples,-1), self.p, 1)

        if self.reduction:
            if self.size_average:
                return torch.mean(all_norms)
            else:
                return torch.sum(all_norms)

        return all_norms

    def rel(self, x, y):
        num_examples = x.size()[0]

        diff_norms = torch.norm(x.reshape(num_examples,-1) - y.reshape(num_examples,-1), self.p, 1)
        y_norms = torch.norm(y.reshape(num_examples,-1), self.p, 1)

        if self.reduction:
            if self.size_average:
                return torch.mean(diff_norms/y_norms)
            else:
                return torch.sum(diff_norms/y_norms)

        return diff_norms/y_norms

    def __call__(self, x, y):
        return self.rel(x, y)

def compute_metrics(y, y_pred, d=1) :
    L2_func = LpLoss(d=d, p=2, reduction=False)
    if y.shape != y_pred.shape :
        raise NotImplementedError
    l2 = L2_func.abs(y, y_pred) # [bs]
    relative_l2 = L2_func.rel(y, y_pred) # [bs]
    mse = F.mse_loss(y_pred, y, reduction='none') # [bs]
    mse = mse.mean(dim=tuple(range(1, mse.ndim)))
    return l2, relative_l2, mse


In [4]:
epochs = 500
lr = 0.001
batch_size = 32

initial_datasize=128
batch_acquire=32
ensemble_size = 5

In [5]:
import torch
import numpy as np
from neuralop.models import FNO
from tqdm import tqdm
import copy

from acquisition.acquirers import select

def train(X_train, Y_train):
    model = FNO(n_modes=cfg.model.n_modes, hidden_channels=64,
                    in_channels=1, out_channels=1)

    model = model.to(device)

    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    criterion = torch.nn.MSELoss()

    dataset = torch.utils.data.TensorDataset(X_train, Y_train)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        for x, y in dataloader:
            # x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
        scheduler.step()
    
    return model

def test(model):
    X_test = Traj_dataset.traj_test[:,0,:].unsqueeze(1).to(device)
    Y_test = Traj_dataset.traj_test[:,-1,:].unsqueeze(1).to(device)

    testset = torch.utils.data.TensorDataset(X_test, Y_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

    model.eval()
    
    Y_test_pred = []
    with torch.no_grad():
        for x, y in testloader:
            # x, y = x.to(device), y.to(device)
            y_pred = model(x)
            Y_test_pred.append(y_pred)
        Y_test_pred = torch.cat(Y_test_pred, dim=0).to(Y_test.device)
    
    metrics = compute_metrics(Y_test, Y_test_pred, d=1)
    return metrics


results = {'datasize': [], 'rel_l2': [], 'l2': [], 'mse': [], 'ensemble': [], 'method': []}

X = Traj_dataset.traj_train[:,0].unsqueeze(1).to(device)
Y = Traj_dataset.traj_train[:,-1].unsqueeze(1).to(device)

train_idxs = torch.arange(initial_datasize, device=device)
pool_idxs = torch.arange(initial_datasize, X.shape[0], device=device)

X_train = X[train_idxs]
Y_train = Y[train_idxs]
X_pool = X[pool_idxs]

ensemble = [train(X_train, Y_train) for _ in tqdm(range(ensemble_size))]

results['datasize'].append(train_idxs.shape[0])
results['ensemble'].append(copy.deepcopy(ensemble))
results['method'].append('random')
l2_list = [test(model)[0].mean().item() for model in ensemble]
rel_l2_list = [test(model)[1].mean().item() for model in ensemble]
mse_list = [test(model)[2].mean().item() for model in ensemble]
results['l2'].append(torch.mean(torch.tensor(l2_list)).item())
results['rel_l2'].append(torch.mean(torch.tensor(rel_l2_list)).item())
results['mse'].append(torch.mean(torch.tensor(mse_list)).item())
print(f'Datasize: {results["datasize"][-1]}, L2: {results["l2"][-1]}, Relative L2: {results["rel_l2"][-1]}, MSE: {results["mse"][-1]}')


100%|██████████| 5/5 [03:22<00:00, 40.58s/it]


Datasize: 128, L2: 1.0003101825714111, Relative L2: 0.0758710503578186, MSE: 0.018993154168128967


In [6]:
for method in ['random', 'variance', 'stochastic', 'lcmd', 'lcmd_shared']:
    print(method)
    time_start = time.time()
    
    X_train = X[train_idxs]
    Y_train = Y[train_idxs]
    X_pool = X[pool_idxs]
    
    new_idxs = select(ensemble, X_train, X_pool, batch_acquire, selection_method=method, device=device)
    # new_idxs = select_var(ensemble, X_pool, batch_acquire)

    new_idxs = new_idxs.to(device)
    print(new_idxs)
    # print(f'{len(new_idxs)=}')
    logical_new_idxs = torch.zeros(pool_idxs.shape[-1], dtype=torch.bool, device=device)
    logical_new_idxs[new_idxs] = True
    train_idxs_new = torch.cat([train_idxs, pool_idxs[logical_new_idxs]], dim=-1)
    pool_idxs_new = pool_idxs[~logical_new_idxs]

    X_train = X[train_idxs_new]
    Y_train = Y[train_idxs_new]

    X_pool = X[pool_idxs_new]

    ensemble_new = [train(X_train, Y_train) for _ in tqdm(range(ensemble_size))]

    results['datasize'].append(train_idxs.shape[0])
    results['ensemble'].append(copy.deepcopy(ensemble_new))
    results['method'].append(method)
    l2_list = [test(model)[0].mean().item() for model in ensemble_new]
    rel_l2_list = [test(model)[1].mean().item() for model in ensemble_new]
    mse_list = [test(model)[2].mean().item() for model in ensemble_new]
    results['l2'].append(torch.mean(torch.tensor(l2_list)).item())
    results['rel_l2'].append(torch.mean(torch.tensor(rel_l2_list)).item())
    results['mse'].append(torch.mean(torch.tensor(mse_list)).item())
    print(f'Datasize: {results["datasize"][-1]}, L2: {results["l2"][-1]}, Relative L2: {results["rel_l2"][-1]}, MSE: {results["mse"][-1]}')

    time_end = time.time()
    print(f'{method} time: {time_end-time_start} s')


random
tensor([216, 387, 593, 121, 300,  97, 852, 496, 487, 402, 515,  76, 727, 867,
        779, 847, 629, 180, 372, 673, 444, 462, 292, 205, 327, 742,  77,  92,
        533, 410, 786, 560], device='cuda:0')


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [04:07<00:00, 49.51s/it]


Datasize: 128, L2: 0.886997401714325, Relative L2: 0.06571687757968903, MSE: 0.016933556646108627
random time: 264.9664704799652 s
variance
tensor([754, 802, 547, 697, 644,  20, 308, 501, 839, 558, 329,  74, 414, 571,
        253,  58, 287, 176, 814, 405, 437, 735, 774, 218, 301, 852, 100, 137,
         95, 407, 728, 652], device='cuda:0')


100%|██████████| 5/5 [04:07<00:00, 49.49s/it]


...

Datasize: 128, L2: 0.8061470985412598, Relative L2: 0.061047106981277466, MSE: 0.012720659375190735
stochastic time: 263.8111062049866 s
lcmd
tensor([754, 839, 501,  20, 253, 414, 547,  58, 558, 587, 449, 735, 571, 680,
        218, 407, 453, 287,   5, 308, 697, 137,  97, 237, 405, 641,  74, 814,
        644, 802, 410, 854], device='cuda:0')


100%|██████████| 5/5 [04:08<00:00, 49.61s/it]


Datasize: 128, L2: 0.8203738331794739, Relative L2: 0.06296046078205109, MSE: 0.012000292539596558
lcmd time: 265.9070177078247 s
lcmd_shared
tensor([ 20, 644, 839, 308,  74, 287, 697, 449, 504, 754, 571, 100, 802, 825,
        776,  58, 814, 218, 414,   5, 652, 694,  63, 253,  95, 854, 176, 558,
        547, 227, 301, 177], device='cuda:0')


100%|██████████| 5/5 [04:08<00:00, 49.68s/it]


Datasize: 128, L2: 0.7886824607849121, Relative L2: 0.060593217611312866, MSE: 0.011279908940196037
lcmd_shared time: 268.18438506126404 s
